In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

In [3]:
!pip install azure-cognitiveservices-vision-computervision
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.2 MB/s eta 0:00:00


In [5]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from matplotlib import pyplot as plt
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image, ImageEnhance, ImageDraw
import sys
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Setarea variabilelor de mediu
os.environ["VISION_KEY"] = "27d682fdc92c400a9aabd9d5aecde91e"
os.environ["VISION_ENDPOINT"] = "https://ggg.cognitiveservices.azure.com/"

# Accesarea cheii și a endpoint-ului din variabilele de mediu
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]

# Crearea clientului ComputerVisionClient
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))


#1
#Sa se foloseasca un algoritm de clasificare a
# imaginilor (etapa de inferenta/testare) si sa se
# stabileasca performanta acestui algoritm de clasificare binara (imagini cu biciclete vs. imagini fara biciclete).

#initializez variabile pt nr date poz si neg
#parcurg img si daca
#Dacă printre etichetele obținute se găsește eticheta "bicycle",
#se decide dacă imaginea este un adevărat pozitiv sau un fals pozitiv în funcție de numele fișierului
#daca nu se descide daca e fals neg sau adv neg
imageDirectory = r"/content/drive/MyDrive/images/bikes"
true_positives = 0
false_positives = 0
false_negatives = 0
true_negatives = 0
for filename in os.listdir(imageDirectory):
    if filename.endswith(".jpg"):
        image_path = os.path.join(imageDirectory, filename)
        with open(image_path, "rb") as image_stream:
            result = computervision_client.analyze_image_in_stream(image_stream, visual_features=[VisualFeatureTypes.tags, VisualFeatureTypes.objects])
            if any(tag.name == 'bicycle' for tag in result.tags):
                if filename.startswith('bike'):
                    true_positives += 1
                else:
                    false_positives += 1
            else:
                if filename.startswith('bike'):
                    false_negatives += 1
                else:
                    true_negatives += 1
#se calc acuratetea precizia si recall
accuracy = (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives)
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
print(accuracy)
print(precision)
print("Recall:", recall)


#2a+2b+2c
#Pentru imaginile care contin biciclete:

#a. sa se localizeze automat bicicletele in aceste imagini
#si sa se evidentieze chenarele care incadreaza bicicletele

#b. sa se eticheteze (fara ajutorul algoritmilor de AI) aceste
# imagini cu chenare care sa incadreze cat mai exact bicicletele. Care
#task dureaza mai mult (cel de la punctul a sau cel de la punctul b)?

#c. sa se determine performanta algoritmului de la punctul
# a avand in vedere etichetarile realizate la punctul b (se vor folosi cel putin 2 metrici).
def calculate_iou(bbox_true, bbox_predicted):
    # Calculează coordonatele dreptunghiului de intersectare
    x_left = max(bbox_true[0], bbox_predicted[0])
    y_top = max(bbox_true[1], bbox_predicted[1])
    x_right = min(bbox_true[2], bbox_predicted[2])
    y_bottom = min(bbox_true[3], bbox_predicted[3])

    # Calculează aria de intersectare
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)

    # Calculează aria de uniune
    area_true = (bbox_true[2] - bbox_true[0]) * (bbox_true[3] - bbox_true[1])
    area_predicted = (bbox_predicted[2] - bbox_predicted[0]) * (bbox_predicted[3] - bbox_predicted[1])
    union_area = area_true + area_predicted - intersection_area

    # Calculează IoU
    iou = intersection_area / union_area if union_area > 0 else 0
    return iou


#1
bike1_bb = [6.0, 33.0, 409.0, 405.0]
img = open("/content/drive/MyDrive/images/bikes/bike1.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike1_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike1.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike1_bb[0], bike1_bb[1]), width=bike1_bb[2] - bike1_bb[0], height=bike1_bb[3] - bike1_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike1_bb[0], predicted_bike1_bb[1]), width=predicted_bike1_bb[2] - predicted_bike1_bb[0],
                  height=predicted_bike1_bb[3] - predicted_bike1_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike1_bb, bike1_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike1_bb, predicted_bike1_bb)
print("Intersection over Union (IoU):", iou)

#2
bike2_bb = [17.8, 90.0, 379.0, 318.0]
img = open("/content/drive/MyDrive/images/bikes/bike02.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike2_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike02.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike2_bb[0], bike2_bb[1]), width=bike2_bb[2] - bike2_bb[0], height=bike2_bb[3] - bike2_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike2_bb[0], predicted_bike2_bb[1]), width=predicted_bike2_bb[2] - predicted_bike2_bb[0],
                  height=predicted_bike2_bb[3] - predicted_bike2_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike2_bb, bike2_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike2_bb, predicted_bike2_bb)
print("Intersection over Union (IoU):", iou)

#3
bike3_bb = [158.8, 147.0, 343.0, 403.0]
img = open("/content/drive/MyDrive/images/bikes/bike03.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike3_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike03.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike3_bb[0], bike3_bb[1]), width=bike3_bb[2] - bike3_bb[0], height=bike3_bb[3] - bike3_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike3_bb[0], predicted_bike3_bb[1]), width=predicted_bike3_bb[2] - predicted_bike3_bb[0],
                  height=predicted_bike3_bb[3] - predicted_bike3_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike3_bb, bike3_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike3_bb, predicted_bike3_bb)
print("Intersection over Union (IoU):", iou)

#4
bike4_bb = [1.0, 0.5, 414.0, 410.0]
img = open("/content/drive/MyDrive/images/bikes/bike04.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike4_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike04.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike4_bb[0], bike4_bb[1]), width=bike4_bb[2] - bike4_bb[0], height=bike4_bb[3] - bike4_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike4_bb[0], predicted_bike4_bb[1]), width=predicted_bike4_bb[2] - predicted_bike4_bb[0],
                  height=predicted_bike4_bb[3] - predicted_bike4_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike4_bb, bike4_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike4_bb, predicted_bike4_bb)
print("Intersection over Union (IoU):", iou)

#5
bike5_bb = [70.0, 51.5, 354.0, 344.0]
img = open("/content/drive/MyDrive/images/bikes/bike05.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike5_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike05.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike5_bb[0], bike5_bb[1]), width=bike5_bb[2] - bike5_bb[0], height=bike5_bb[3] - bike5_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike5_bb[0], predicted_bike5_bb[1]), width=predicted_bike5_bb[2] - predicted_bike5_bb[0],
                  height=predicted_bike5_bb[3] - predicted_bike5_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike5_bb, bike5_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike5_bb, predicted_bike5_bb)
print("Intersection over Union (IoU):", iou)

#6
bike6_bb = [158.8, 147.0, 343.0, 403.0]
img = open("/content/drive/MyDrive/images/bikes/bike06.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
predicted_bike6_bb=[0,0,0,0]
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike6_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike06.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike6_bb[0], bike6_bb[1]), width=bike6_bb[2] - bike6_bb[0], height=bike6_bb[3] - bike6_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike6_bb[0], predicted_bike6_bb[1]), width=predicted_bike6_bb[2] - predicted_bike6_bb[0],
                  height=predicted_bike6_bb[3] - predicted_bike6_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike6_bb, bike6_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike6_bb, predicted_bike6_bb)
print("Intersection over Union (IoU):", iou)

#7
bike7_bb = [61.0, 205.5, 297.0, 414.0]
img = open("/content/drive/MyDrive/images/bikes/bike07.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike7_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike07.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike7_bb[0], bike7_bb[1]), width=bike7_bb[2] - bike7_bb[0], height=bike7_bb[3] - bike7_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike7_bb[0], predicted_bike7_bb[1]), width=predicted_bike7_bb[2] - predicted_bike7_bb[0],
                  height=predicted_bike7_bb[3] - predicted_bike7_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike7_bb, bike7_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike7_bb, predicted_bike7_bb)
print("Intersection over Union (IoU):", iou)

#8
bike8_bb = [54.5, 1.1, 386.0, 352.0]
img = open("/content/drive/MyDrive/images/bikes/bike08.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
predicted_bike8_bb=[0,0,0,0]
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike8_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike08.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike8_bb[0], bike8_bb[1]), width=bike8_bb[2] - bike8_bb[0], height=bike8_bb[3] - bike8_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike8_bb[0], predicted_bike8_bb[1]), width=predicted_bike8_bb[2] - predicted_bike8_bb[0],
                  height=predicted_bike8_bb[3] - predicted_bike8_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike8_bb, bike8_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike8_bb, predicted_bike8_bb)
print("Intersection over Union (IoU):", iou)

#9
bike9_bb = [4.3, 9.5, 378.6, 382.0]
img = open("/content/drive/MyDrive/images/bikes/bike09.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike9_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike09.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike9_bb[0], bike9_bb[1]), width=bike9_bb[2] - bike9_bb[0], height=bike9_bb[3] - bike9_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike9_bb[0], predicted_bike9_bb[1]), width=predicted_bike9_bb[2] - predicted_bike9_bb[0],
                  height=predicted_bike9_bb[3] - predicted_bike9_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike9_bb, bike9_bb):
    err = err + (v[0] - v[1]) ** 2
    err /= 4
print("detection error: ", err)
iou = calculate_iou(bike9_bb, predicted_bike9_bb)
print("Intersection over Union (IoU):", iou)


#10
bike10_bb = [143.0, 126.0, 373.0, 402.0]
img = open("/content/drive/MyDrive/images/bikes/bike10.jpg", "rb")
result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
for ob in result.objects:
    if (ob.object_property == "bike" or ob.object_property == "bicycle"):
        predicted_bike10_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w,
                            ob.rectangle.y + ob.rectangle.h]
im = plt.imread("/content/drive/MyDrive/images/bikes/bike10.jpg")
fig = plt.imshow(im)
fig.axes.add_patch(
    plt.Rectangle(xy=(bike10_bb[0], bike10_bb[1]), width=bike10_bb[2] - bike10_bb[0], height=bike10_bb[3] - bike10_bb[1], fill=False,
                  color="red", linewidth=2))
fig.axes.add_patch(
    plt.Rectangle(xy=(predicted_bike10_bb[0], predicted_bike10_bb[1]), width=predicted_bike10_bb[2] - predicted_bike10_bb[0],
                  height=predicted_bike10_bb[3] - predicted_bike10_bb[1], fill=False, color="green", linewidth=2))
plt.show()
err = 0
for v in zip(predicted_bike10_bb, bike10_bb):
    err = err + (v[0] - v[1]) ** 2
err /= 4
print("detection error: ", err)
iou = calculate_iou(bike10_bb, predicted_bike10_bb)
print("Intersection over Union (IoU):", iou)

Output hidden; open in https://colab.research.google.com to view.